## Stereo vision calibration
Dataset: https://www.kaggle.com/datasets/danielwe14/stereocamera-chessboard-pictures

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import glob
import sys

if '/Users/damianstone/Documents/Code/tennis-project/post-triangulation/' not in sys.path:
  sys.path.append('/Users/damianstone/Documents/Code/tennis-project/post-triangulation/')


In [2]:
def print_image(imread_img):
  img = cv.cvtColor(imread_img, cv.COLOR_BGR2RGB)
  plt.imshow(img)
  plt.axis('off')
  plt.show()

### Find chessboard corners - object points and image points


In [11]:
# number of inner coners - rows and columns
chessboard_size = (11, 7)

img = cv.imread('/Users/damianstone/Documents/Code/tennis-project/post-triangulation/images/stereoRight/Im_R_1.png')
height, width = img.shape[:2]

# size of the image
frame_size = (width, height)

# how exact we want to be find the corners of each square in the chessboard 
# 30 = max number of iterations
# 0.001 = stop if the change in the corner position is less than 0.001 - convergence threshold 
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# create 3D points of the chessboard corners
# each row represents a coordinate of a corner of the chessboard
# corner = interesection between two squares
objp = np.zeros(
  (chessboard_size[0] * chessboard_size[1], 3), 
  np.float32
)

# it generates a grid of (x, y) coordinates for chessboard corners and stores them as 3D points with z = 0 for calibration
objp[:,:2] = np.mgrid[
  0:chessboard_size[0],
  0:chessboard_size[1]
  ].T.reshape(-1,2)

# scale the 3D points to real world size
size_of_chessboard_squares_mm = 30
objp = objp * size_of_chessboard_squares_mm

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints_l = [] # 2d points in image plane
imgpoints_r = [] # 2d points in image plane


images_left = glob.glob('/Users/damianstone/Documents/Code/tennis-project/post-triangulation/images/stereoLeft/*.png')
images_right = glob.glob('/Users/damianstone/Documents/Code/tennis-project/post-triangulation/images/stereoRight/*.png')

for img_left, img_right in zip(images_left, images_right):
  img_l = cv.imread(img_left)
  img_r = cv.imread(img_right)

  gray_l = cv.cvtColor(img_l, cv.COLOR_BGR2GRAY)
  gray_r = cv.cvtColor(img_r, cv.COLOR_BGR2GRAY)

  # find the chessboard corners
  ret_l, corners_l = cv.findChessboardCorners(gray_l, chessboard_size, None)
  ret_r, corners_r = cv.findChessboardCorners(gray_r, chessboard_size, None)

  # if corners are found, add object points, image points (after refining them)
  if ret_l and ret_r:
    objpoints.append(objp)

    corners_l = cv.cornerSubPix(gray_l, corners_l, (11, 11), (-1, -1), criteria)
    imgpoints_l.append(corners_l)

    corners_r = cv.cornerSubPix(gray_r, corners_r, (11, 11), (-1, -1), criteria)
    imgpoints_r.append(corners_r)

    # draw and display the corners
    # cv.drawChessboardCorners(img_l, chessboard_size, corners_l, ret_l)
    # print_image(img_l)
    # cv.drawChessboardCorners(img_r, chessboard_size, corners_r, ret_r)
    # print_image(img_r)
    cv.waitKey(1000)

cv.destroyAllWindows()



### Calibration

In [ ]:
ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
pickle.dump((cameraMatrix, dist), open( "calibration.pkl", "wb" ))
pickle.dump(cameraMatrix, open( "cameraMatrix.pkl", "wb" ))
pickle.dump(dist, open( "dist.pkl", "wb" ))


### Undistortion

In [ ]:
img = cv.imread('cali5.png')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

# Undistort
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult1.png', dst)

# Undistort with Remapping
mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult2.png', dst)


# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "total error: {}".format(mean_error/len(objpoints)) )